# Pre-processing

This notebook contains pre-processing steps to create cleaned data in `/data` from the raw datasets downloaded and stored in `/raw_data`. Before running this notebook, please ensure that you ran `src/scripts/download.py` to download the raw data.

In [2]:
import pandas as pd
import json
import os
import subprocess
from thefuzz import process

In [3]:
os.makedirs('data', exist_ok=True)
os.makedirs('data/beer_advocate', exist_ok=True)
os.makedirs('data/matched_beer_data', exist_ok=True)
os.makedirs('data/rate_beer', exist_ok=True)

## Functions

### Locations

In [3]:
# Map region to country (useful when we only have the region)
with open('src/utils/region_to_country.json', 'r') as region_to_country_file:
    REGION_TO_COUNTRY = json.load(region_to_country_file)

In [4]:
# Assign unique code to each country
df_iso_codes = pd.read_csv('src/utils/iso_codes.csv')[['name', 'alpha-3']]

In [5]:
# Try to get the country code from the country using fuzzy matching
def get_closest_match_or_none(query: str, serie_choices: pd.Series, serie_values: pd.Series, threshold: int = 80):
    match, score, _ = process.extractOne(query, serie_choices)
    if score >= threshold:
        return serie_values[serie_choices == match].values[0]
    return None

In [6]:
def format_location(serie_locations: pd.Series):
    serie_locations.replace({
        'UNKNOWN': pd.NA,
        'Swaziland': 'Esawtini',                                                # Swaziland changed its name to Esawtini
        'Northern Ireland': 'United Kingdom, Northern Ireland',                 # Northern Ireland is part of the United Kingdom
        'Aotearoa': 'New Zealand',                                              # Aotearoa is the Maori name for New Zealand
        'Nagorno-Karabakh': 'Azerbaijan',                                       # Nagorno-Karabakh is a disputed territory (currenlty controlled by Azerbaijan)
        'Transdniestra': 'Moldova',                                             # Transdniestra is a disputed territory (currenlty controlled by Moldova)
        'Tibet': 'China',                                                       # Tibet is an autonomous region of China
        'Abkhazia': 'Georgia',                                                  # Abkhazia is a disputed territory (currenlty controlled by Georgia)
    }, inplace=True)

    # Match location with Google Maps links and extract the location (before the </a> tag)
    extracted_location_matches = serie_locations.str.extract(r'^(.*?)(?=</a>)|^(.*)$')
    serie_locations = extracted_location_matches[0].fillna(extracted_location_matches[1])   

    splits = serie_locations.str.split(', ', expand=True)
    splits.rename(columns={0: 'country', 1: 'region'}, inplace=True)

    # Replace occurence of regions that were considered as countries
    splits['country'] = splits['country'].replace(REGION_TO_COUNTRY)

    # Try to match the country with the ISO code (exact match)
    splits = splits.merge(df_iso_codes[['name', 'alpha-3']], left_on='country', right_on='name', how='left')

    # Try to match the country with ISO code (fuzzy match)
    splits['alpha-3'] = splits.apply(
        lambda row: row['alpha-3'] if pd.notna(row['alpha-3']) or pd.isna(row['country']) else 
        get_closest_match_or_none(
            row['country'],
            df_iso_codes['name'],
            df_iso_codes['alpha-3']
        ), axis=1
    )

    splits.loc[splits['country'] == 'Kosovo', 'alpha-3'] = 'XKX' # Kosovo is not in the ISO list
    splits.loc[splits['country'] == 'Vatican City', 'alpha-3'] = 'VAT' # Vatican City is not in the ISO list

    return splits['country'], splits['region'], splits['alpha-3']

## Beer Advocate

### Beers

In [7]:
df_ba_beers = pd.read_csv('raw_data/beer_advocate/beers.csv')
df_ba_beers_styles = pd.read_csv('src/utils/beers_styles.csv')

In [8]:
df_ba_beers.rename(columns={
    'style': 'beer_style',
    'nbr_ratings': 'ratings_count',
    'nbr_reviews': 'reviews_count',
    'avg': 'ratings_average',
    'ba_score': 'ratings_ba_score',
    'bros_score': 'ratings_bros_score',
    'abv': 'beer_alcohol_by_volume',
    'avg_computed': 'ratings_average_computed',
    'nbr_matched_valid_ratings': 'matching_ratings_count',
    'avg_matched_valid_ratings': 'matching_ratings_average',
}, inplace=True)

df_ba_beers.drop(columns=['brewery_name'], inplace=True, errors='ignore')

df_ba_beers = df_ba_beers[['beer_id', 'brewery_id', 'beer_name', 'beer_style', 'ratings_count', 'reviews_count', 'ratings_average', 'ratings_ba_score', 'ratings_bros_score', 'beer_alcohol_by_volume', 'ratings_average_computed', 'zscore', 'matching_ratings_count', 'matching_ratings_average']]

In [9]:
df_ba_beers = df_ba_beers.merge(df_ba_beers_styles, on='beer_style', how='left')

In [10]:
df_ba_beers.to_csv('data/beer_advocate/beers.csv', index=False)

### Breweries

In [11]:
df_ba_breweries = pd.read_csv('raw_data/beer_advocate/breweries.csv')

In [12]:
df_ba_breweries.rename(columns={
    'id': 'brewery_id',
    'location': 'brewery_location',
    'name': 'brewery_name',
    'nbr_beers': 'brewery_beers_count',
}, inplace=True)

In [13]:
# Location of two breweries were missing, we found their location using Google
df_ba_breweries.loc[df_ba_breweries['brewery_id'] == 18989, 'brewery_location'] = 'United States'
df_ba_breweries.loc[df_ba_breweries['brewery_id'] == 11016, 'brewery_location'] = 'Austria'

In [14]:
df_ba_breweries['brewery_country'], df_ba_breweries['brewery_region'], df_ba_breweries['brewery_country_code'] = format_location(df_ba_breweries['brewery_location'])
df_ba_breweries.drop(columns=['brewery_location'], inplace=True)

In [15]:
df_ba_breweries.to_csv('data/beer_advocate/breweries.csv', index=False)

### Users

In [16]:
df_ba_users = pd.read_csv('raw_data/beer_advocate/users.csv')

In [17]:
df_ba_users.rename(columns={
    'joined': 'user_created_date',
    'location': 'user_location',
    'nbr_ratings': 'user_ratings_count',
    'nbr_reviews': 'user_reviews_count',
}, inplace=True)

In [18]:
df_ba_users['user_country'], df_ba_users['user_region'], df_ba_users['user_country_code'] = format_location(df_ba_users['user_location'])
df_ba_users.drop(columns=['user_location'], inplace=True)

In [19]:
df_ba_users.to_csv('data/beer_advocate/users.csv', index=False)

### Ratings & Reviews

On BeerAdvocate, users can either submit a *rating* or a *review*. The dataset contains two separate text files `ratings.txt` and `reviews.txt`. Ratings and reviews are formatted as (key, value) pairs on each line of a plain text file with empty lines to split different ratings/reviews. 

The program `txt_to_csv.cpp` is a C++ program that transform a plain text file in a `.csv` file, that can be more easily analyzed using Python and Pandas. The following cell will compile the program using a C++ compiler and run it on the `ratings.txt` file. Indeed, after analyzing the overlapping between the two files, we found that `reviews.txt` is a subset of `ratings.txt`. Thus, we will discard `reviews.txt` and only keep `ratings.txt`.

In [ ]:
subprocess.run('g++ -std=c++17 -o src/scripts/txt_to_csv src/scripts/txt_to_csv.cpp', shell=True)

subprocess.run([
    'src/scripts/txt_to_csv', 
    'raw_data/beer_advocate/ratings.txt', 
    'raw_data/beer_advocate/ratings.csv'
])

In [88]:
df_ba_ratings = pd.read_csv('raw_data/beer_advocate/ratings.csv')

In [89]:
# Those columns are likely to have already been deleted by the C++ script, hence the errors='ignore' parameter
df_ba_ratings.drop(columns=['user_name', 'brewery_name', 'beer_name', 'style', 'abv'], inplace=True, errors='ignore')
df_ba_ratings = df_ba_ratings[['user_id', 'beer_id', 'brewery_id', 'date', 'review', 'rating', 'overall', 'aroma', 'appearance', 'palate', 'taste', 'text']]

In [90]:
df_ba_ratings = df_ba_ratings.sort_values(['user_id', 'date'])

df_ba_ratings['user_past_ratings_count']   = df_ba_ratings.groupby('user_id').cumcount()
df_ba_ratings['user_past_ratings_average'] = df_ba_ratings.groupby('user_id')['rating'].expanding().mean().shift().reset_index(level=0, drop=True)

df_ba_ratings.loc[df_ba_ratings['user_past_ratings_count'] == 0, 'user_past_ratings_average'] = pd.NA

In [ ]:
df_ba_ratings = df_ba_ratings.sort_values(['beer_id', 'date'])

df_ba_ratings['beer_past_ratings_count']   = df_ba_ratings.groupby('beer_id').cumcount()
df_ba_ratings['beer_past_ratings_average'] = df_ba_ratings.groupby('beer_id')['rating'].expanding().mean().reset_index(level=0).groupby('beer_id')['rating'].shift()

In [96]:
df_ba_ratings.to_csv('data/beer_advocate/ratings.csv', index=False)

### Memory cleaning

In [ ]:
del df_ba_beers
del df_ba_breweries
del df_ba_users
del df_ba_ratings

## RateBeer

### Beers

In [ ]:
df_rb_beers = pd.read_csv('raw_data/rate_beer/beers.csv')

df_rb_beers = df_rb_beers.drop(columns=['brewery_name'])

df_rb_beers = df_rb_beers.rename(columns={
    'style': 'beer_style',
    'nbr_ratings': 'ratings_count',
    'avg': 'ratings_average',
    'overall_score': 'ratings_overall_score',
    'abv': 'beer_alcohol_by_volume',
    'avg_computed': 'ratings_average_computed',
    'nbr_matched_valid_ratings': 'matching_ratings_count',
    'avg_matched_valid_ratings': 'matching_ratings_average',
})

df_rb_beers.to_csv('data/rate_beer/beers.csv', index=False)

### Breweries

In [ ]:
df_rb_breweries = pd.read_csv('raw_data/rate_beer/breweries.csv')

df_rb_breweries = df_rb_breweries.rename(columns={
    'id': 'brewery_id',
    'location': 'brewery_location',
    'name': 'brewery_name',
    'nbr_beers': 'brewery_beers_count',
})

In [ ]:
df_rb_breweries['brewery_country'], df_rb_breweries['brewery_region'], df_rb_breweries['brewery_country_code'] = format_location(df_rb_breweries['brewery_location'])
df_rb_breweries.drop(columns=['brewery_location'], inplace=True)

In [ ]:
df_rb_breweries.to_csv('data/rate_beer/breweries.csv', index=False)

### Users

In [ ]:
df_rb_users = pd.read_csv('raw_data/rate_beer/users.csv')

In [ ]:
df_rb_users = df_rb_users.groupby('user_id').agg({
    'nbr_ratings': 'sum',
    'user_name': 'first',
    'joined': 'min',
    'location': 'first',
}).reset_index()

In [ ]:
df_rb_users = df_rb_users.rename(columns={
    'joined': 'user_created_date',
    'location': 'user_location',
    'nbr_ratings': 'user_ratings_count',
})

In [ ]:
df_rb_users['user_country'], df_rb_users['user_region'], df_rb_users['user_country_code'] = format_location(df_rb_users['user_location'])
df_rb_users = df_rb_users.drop(columns=['user_location'])

In [ ]:
df_rb_users.to_csv('data/rate_beer/users.csv', index=False)

### Ratings & Reviews

Since there are only ratings on RB, the files `ratings.txt` and `reviews.txt` are strictly identical. We discard `reviews.txt` and parse `ratings.txt` using the `txt_to_csv.cpp` utility.

In [ ]:
subprocess.run([
    'src/scripts/txt_to_csv', 
    'raw_data/rate_beer/ratings.txt', 
    'raw_data/rate_beer/ratings.csv'
])

In [ ]:
df_rb_ratings = pd.read_csv('raw_data/rate_beer/ratings.csv')
df_rb_ratings = df_rb_ratings.groupby(['user_id', 'beer_id']).last().reset_index()
df_rb_ratings = df_rb_ratings[['user_id', 'beer_id', 'brewery_id', 'date', 'rating', 'overall', 'aroma', 'appearance', 'palate', 'taste', 'text']]
df_rb_ratings.to_csv('data/rate_beer/ratings.csv', index=False)

### Memory cleaning

In [ ]:
del df_rb_beers
del df_rb_breweries
del df_rb_users
del df_rb_ratings

## Matched Beer Data

### Beers

In [ ]:
df_ma_beers = pd.read_csv('raw_data/matched_beer_data/beers.csv', header=[0, 1])
df_ma_beers = df_ma_beers[[('ba', 'beer_id'), ('rb', 'beer_id'), ('scores', 'diff'), ('scores', 'sim')]]
df_ma_beers.to_csv('data/matched_beer_data/beers.csv', index=False)

### Breweries

In [ ]:
df_ma_breweries = pd.read_csv('raw_data/matched_beer_data/breweries.csv', header=[0, 1])
df_ma_breweries = df_ma_breweries.rename(columns={'id': 'brewery_id',}, level=1)
df_ma_breweries = df_ma_breweries[[('ba', 'brewery_id'), ('rb', 'brewery_id'), ('scores', 'diff'), ('scores', 'sim')]]
df_ma_breweries.to_csv('data/matched_beer_data/breweries.csv', index=False)

### Users

In [ ]:
df_ma_users = pd.read_csv('raw_data/matched_beer_data/users.csv', header=[0, 1])
df_ma_users = df_ma_users[[('ba', 'user_id'), ('rb', 'user_id')]]
df_ma_users.to_csv('data/matched_beer_data/users.csv', index=False)

In [ ]:
df_ma_users_approx = pd.read_csv('raw_data/matched_beer_data/users_approx.csv', header=[0, 1])
df_ma_users_approx = df_ma_users_approx[[('ba', 'user_id'), ('rb', 'user_id'), ('scores', 'sim')]]
df_ma_users_approx.to_csv('data/matched_beer_data/users_approx.csv', index=False)

### Ratings & Reviews

In [ ]:
df_ma_ratings = pd.read_csv('raw_data/matched_beer_data/ratings.csv', header=[0, 1])
df_ma_ratings = df_ma_ratings[[
    ('ba', 'beer_id'), ('ba', 'user_id'), 
    ('rb', 'beer_id'), ('rb', 'user_id'),
]]
df_ma_ratings.to_csv('data/matched_beer_data/ratings.csv', index=False)

### Memory cleaning

In [ ]:
del df_ma_beers
del df_ma_breweries
del df_ma_users
del df_ma_users_approx
del df_ma_ratings